In [ ]:
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm, trange
from torch import nn

In [ ]:
import torch
from transformers import BertModel

if torch.cuda.is_available():
    device = torch.cuda.current_device()
    print('Current device:', torch.cuda.get_device_name(device))
    torch.cuda.empty_cache()
else:
    print('Failed to find GPU. Will use CPU.')
    device = 'cpu'

In [ ]:
from utils.Constants import tokenizer, PRE_TRAINED_MODEL_NAME
from dataLoader.DataLoader import get_data_loader

max_test_size = 100
test_data_loader = get_data_loader(data_type = 'test', tokenizer = tokenizer, batch_size = 1, max_size=max_test_size)

In [ ]:
from models.QABert import QABertTrainer
from models.SelectRanker import SelectRankerTrainer
from models.WhereRanker import WhereRankerTrainer
from models.WhereConditionClassifier import WhereConditionClassifierTrainer
from models.WhereNumberClassifier import WhereNumberClassifierTrainer
from models.AggregationClassifier import AggregationClassifierTrainer

from train.Trainer import load_model

bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
for param in bert.parameters():
    param.requires_grad = False

path=None

if path:
    models = load_model(path, test_data_loader, device)
else:
    models = dict(
        selection_trainer = SelectRankerTrainer(device, test_data_loader, bert),
        agg_class_trainer = AggregationClassifierTrainer(device, test_data_loader, bert, use_pretrained=False),
        where_ranker_trainer = WhereRankerTrainer(device, test_data_loader, bert),
        where_cond_class_trainer = WhereConditionClassifierTrainer(device, test_data_loader, bert, use_pretrained=False),
        where_numb_class_trainer = WhereNumberClassifierTrainer(device, test_data_loader, bert),
        qa_trainer = QABertTrainer(device, test_data_loader, bert, use_pretrained=False),
    )

In [ ]:
from train.Trainer import get_eval

eval = get_eval(models, test_data_loader, max_test_size, device)

print(eval)
